# FastPitch MultiSpeaker Pretraining

This notebook is designed to provide a guide on how to run FastPitch MultiSpeaker Pretraining Pipeline. It contains the following sections:
1. **Pre-train FastPitch on multi-speaker data**: pre-train a multi-speaker FastPitch
* Dataset Preparation: download dataset and extract manifest files.
* Preprocessing: add absolute audio paths in manifest, calibrate speaker id to start from 0, and extract Supplementary Data.
* Training: pre-train multispeaker FastPitch
2. **Fine-tune HiFiGAN on multi-speaker data**: fine-tune a vocoder for the pre-trained multi-speaker FastPitch
* Dataset Preparation: extract mel-spectrograms from pre-trained FastPitch.
* Training: fine-tune HiFiGAN with pre-trained multi-speaker data.
3. **Inference**: generate speech from pre-trained multi-speaker FastPitch
* Load Model: load pre-trained multi-speaker FastPitch.
* Output Audio: generate audio files.

# License
> Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""
# BRANCH = 'r1.22.0'
# # If you're using Colab and not running locally, uncomment and run this cell.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode pynini==2.1.4 scipy==1.7.3
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# # Download local version of NeMo scripts. If you are running locally and want to use your own local NeMo code,
# # comment out the below lines and set `code_dir` to your local path.
code_dir = 'NeMoTTS' 
!git clone https://github.com/NVIDIA/NeMo.git {code_dir}

In [ ]:
!wandb login #PASTE_WANDB_APIKEY_HERE

In [ ]:
sample_rate = 44100
# Store all manifest and audios
data_dir = 'NeMoTTS_dataset'
# Store all supplementary files
supp_dir = "NeMoTTS_sup_data"
# Store all training logs
logs_dir = "NeMoTTS_logs"
# Store all mel-spectrograms for vocoder training
mels_dir = "NeMoTTS_mels"

In [ ]:
import os
import json
import nemo
import torch
import numpy as np

from pathlib import Path
from tqdm import tqdm

In [ ]:
os.makedirs(code_dir, exist_ok=True)
code_dir = os.path.abspath(code_dir)
os.makedirs(data_dir, exist_ok=True)
data_dir = os.path.abspath(data_dir)
os.makedirs(supp_dir, exist_ok=True)
supp_dir = os.path.abspath(supp_dir)
os.makedirs(logs_dir, exist_ok=True)
logs_dir = os.path.abspath(logs_dir)
os.makedirs(mels_dir, exist_ok=True)
mels_dir = os.path.abspath(mels_dir)

# 1. Pre-train FastPitch on multi-speaker data

## a. Dataset Preparation
For our tutorial, we use the subset of VCTK dataset with 5 speakers (p225-p229). The audios have 48 kHz sampling rate, we downsample to 44.1 kHz in this tutorial. 
You can read more about dataset [here](https://datashare.ed.ac.uk/handle/10283/2950).

In [ ]:
!cd {data_dir} && wget https://vctk-subset.s3.amazonaws.com/vctk_subset_multispeaker.tar.gz && tar zxf vctk_subset_multispeaker.tar.gz

In [ ]:
manidir = f"{data_dir}/vctk_subset_multispeaker"
!ls {manidir}

In [ ]:
train_manifest = os.path.abspath(os.path.join(manidir, 'train.json'))
valid_manifest = os.path.abspath(os.path.join(manidir, 'dev.json'))

## b. Preprocessing

### Add absolute audio path in manifest
We use absolute path for `audio_filepath` to get the audio during training.

In [ ]:
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest

In [ ]:
train_data = read_manifest(train_manifest)
for m in train_data: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
write_manifest(train_manifest, train_data)

valid_data = read_manifest(valid_manifest)
for m in valid_data: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
write_manifest(valid_manifest, valid_data)

### Calibrate speaker id to start from 0
We use speaker id start from 0, so we can create a speaker look-up table with speaker size.

In [ ]:
train_data = read_manifest(train_manifest)
speaker2id = {s: _id for _id, s in enumerate(set([m['speaker'] for m in train_data]))}
for m in train_data: m['old_speaker'], m['speaker'] = m['speaker'], speaker2id[m['speaker']]
write_manifest(train_manifest, train_data)

valid_data = read_manifest(valid_manifest)
for m in valid_data: m['old_speaker'], m['speaker'] = m['speaker'], speaker2id[m['speaker']]
write_manifest(valid_manifest, valid_data)

### Extract Supplementary Data

As mentioned in the [FastPitch and MixerTTS training tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_MixerTTS_Training.ipynb) - To accelerate and stabilize our training, we also need to extract pitch for every audio, estimate pitch statistics (mean, std, min, and max). To do this, all we need to do is iterate over our data one time, via `extract_sup_data.py` script.

In [ ]:
!cd {code_dir} && python scripts/dataset_processing/tts/extract_sup_data.py \
    manifest_filepath={train_manifest} \
    sup_data_path={supp_dir} \
    dataset.sample_rate={sample_rate} \
    dataset.n_fft=2048 \
    dataset.win_length=2048 \
    dataset.hop_length=512

After running the above command line, you will observe a new folder NeMoTTS_sup_data/pitch and printouts of pitch statistics like below. Specify these values to the FastPitch training configurations. We will be there in the following section.
```bash
PITCH_MEAN=140.84278869628906, PITCH_STD=50.97673034667969
PITCH_MIN=65.4063949584961, PITCH_MAX=285.3046875
```

In [ ]:
!cd {code_dir} && python scripts/dataset_processing/tts/extract_sup_data.py \
    manifest_filepath={valid_manifest} \
    sup_data_path={supp_dir} \
    dataset.sample_rate={sample_rate} \
    dataset.n_fft=2048 \
    dataset.win_length=2048 \
    dataset.hop_length=512

* If you want to compute pitch mean and std for each speaker, you can use the script `compute_speaker_stats.py`
```bash
!cd {code_dir} && python scripts/dataset_processing/tts/compute_speaker_stats.py \
    --manifest_path={train_manifest} \
    --sup_data_path={supp_dir} \
    --pitch_stats_path={data_dir}/pitch_stats.json
```

## c. Training

In [ ]:
phoneme_dict_path = os.path.abspath(os.path.join(code_dir, "scripts", "tts_dataset_files", "cmudict-0.7b_nv22.10"))
heteronyms_path = os.path.abspath(os.path.join(code_dir, "scripts", "tts_dataset_files", "heteronyms-052722"))

# Copy and Paste the PITCH_MEAN and PITCH_STD from previous steps (train_manifest) to override pitch_mean and pitch_std configs below.
PITCH_MEAN=140.84278869628906
PITCH_STD=65.4063949584961

### Important notes
* `sup_data_types="['align_prior_matrix', 'pitch', 'speaker_id', 'reference_audio']" `
    * **speaker_id**: each data has an unique speaker index (start from 0) in the input.
    * **reference_audio**: each data has a reference audio (from the same speaker) in the input.
    
* `model.speaker_encoder.lookup_module.n_speakers`
    * if use **model.speaker_encoder.lookup_module**, please give n_speakers to create the lookup table

* `condition_types="['add', 'concat', layernorm']`
    * use different operation type to condition module (e.g. input_fft/output_fft/duration_predictor/pitch_predictor/alignment_module)
    * **add**: add conditions to module input
    * **concat**: concat conditions to module input
    * **layernorm**: scale and shift layernorm outputs based on conditions
    
* Other default arguments in config:
    * `model.speaker_encoder.lookup_module`: model creates lookup table to get speaker embedding from speaker id.
    * `model.speaker_encoder.gst_module`: model creates global style token to extract speaker information from reference audio.

* Other optional arguments based on your preference:
    * batch_size
    * max_duration
    * min_duration
    * exp_manager
    * trainer

In [ ]:
# Normally 200 epochs
!(cd {code_dir} && python examples/tts/fastpitch.py \
--config-name=fastpitch_align_44100_adapter.yaml \
+init_from_pretrained_model="tts_en_fastpitch" \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
sup_data_types="['align_prior_matrix', 'pitch', 'speaker_id', 'reference_audio']" \
sup_data_path={supp_dir} \
pitch_mean={PITCH_MEAN} \
pitch_std={PITCH_STD} \
phoneme_dict_path={phoneme_dict_path} \
heteronyms_path={heteronyms_path} \
model.speaker_encoder.lookup_module.n_speakers=5 \
model.input_fft.condition_types="['add', 'layernorm']" \
model.output_fft.condition_types="['add', 'layernorm']" \
model.duration_predictor.condition_types="['add', 'layernorm']" \
model.pitch_predictor.condition_types="['add', 'layernorm']" \
model.alignment_module.condition_types="['add']" \
model.train_ds.dataloader_params.batch_size=8 \
model.validation_ds.dataloader_params.batch_size=8 \
model.train_ds.dataset.max_duration=20 \
model.validation_ds.dataset.max_duration=20 \
model.validation_ds.dataset.min_duration=0.1 \
exp_manager.exp_dir={logs_dir} \
+exp_manager.create_wandb_logger=True \
+exp_manager.wandb_logger_kwargs.name="tutorial-FastPitch-pretrain-multispeaker" \
+exp_manager.wandb_logger_kwargs.project="NeMo" \
trainer.max_epochs=20 \
trainer.check_val_every_n_epoch=20 \
trainer.log_every_n_steps=1 \
trainer.devices=-1 \
trainer.strategy=ddp \
trainer.precision=32 \
)

In [ ]:
# e.g. NeMoTTS_logs/FastPitch/Y-M-D_H-M-S/checkpoints/FastPitch.nemo
last_checkpoint_dir = sorted(list([i for i in (Path(logs_dir) / "FastPitch").iterdir() if i.is_dir()]))[-1] / "checkpoints"
pretrained_fastpitch_checkpoint = os.path.abspath(list(last_checkpoint_dir.glob('*.nemo'))[0])
print(pretrained_fastpitch_checkpoint)

# 2. Fine-tune HiFiGAN on multi-speaker data

## a. Dataset Preparation
Generate mel-spectrograms for HiFiGAN training.

In [ ]:
!cd {code_dir} \
&& python scripts/dataset_processing/tts/resynthesize_dataset.py \
--model-path={pretrained_fastpitch_checkpoint} \
--input-json-manifest={train_manifest} \
--input-sup-data-path={supp_dir} \
--output-folder={mels_dir} \
--device="cuda:0" \
--batch-size=1 \
--num-workers=1 \
&& python scripts/dataset_processing/tts/resynthesize_dataset.py \
--model-path={pretrained_fastpitch_checkpoint} \
--input-json-manifest={valid_manifest} \
--input-sup-data-path={supp_dir} \
--output-folder={mels_dir} \
--device="cuda:0" \
--batch-size=1 \
--num-workers=1

In [ ]:
train_manifest_mel = f"{mels_dir}/train_mel.json"
valid_manifest_mel = f"{mels_dir}/dev_mel.json"

## b. Training

In [ ]:
# Normally 100 epochs
!cd {code_dir} && python examples/tts/hifigan_finetune.py \
--config-name=hifigan_44100.yaml \
train_dataset={train_manifest_mel} \
validation_datasets={valid_manifest_mel} \
+init_from_pretrained_model="tts_en_hifitts_hifigan_ft_fastpitch" \
model.train_ds.dataloader_params.batch_size=32 \
model.optim.lr=0.0001 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune \
+trainer.max_epochs=5 \
trainer.check_val_every_n_epoch=5 \
trainer.devices=1 \
trainer.strategy='ddp_find_unused_parameters_true' \
trainer.precision=16 \
exp_manager.exp_dir={logs_dir} \
exp_manager.create_wandb_logger=True \
exp_manager.wandb_logger_kwargs.name="tutorial-HiFiGAN-finetune-multispeaker" \
exp_manager.wandb_logger_kwargs.project="NeMo"

In [ ]:
# e.g. NeMoTTS_logs/HifiGan/Y-M-D_H-M-S/checkpoints/HifiGan.nemo
last_checkpoint_dir = sorted(list([i for i in (Path(logs_dir) / "HifiGan").iterdir() if i.is_dir()]))[-1] / "checkpoints"
finetuned_hifigan_on_multispeaker_checkpoint = os.path.abspath(list(last_checkpoint_dir.glob('*.nemo'))[0])
finetuned_hifigan_on_multispeaker_checkpoint

# 3. Inference

In [ ]:
from nemo.collections.asr.parts.preprocessing.features import WaveformFeaturizer
from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from collections import defaultdict
import IPython.display as ipd
import matplotlib.pyplot as plt

## a. Load Model

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=sample_rate)

In [ ]:
# FastPitch
spec_model = FastPitchModel.restore_from(pretrained_fastpitch_checkpoint).eval().cuda()

In [ ]:
# HiFiGAN
vocoder_model = HifiGanModel.restore_from(finetuned_hifigan_on_multispeaker_checkpoint).eval().cuda()

## b. Output Audio

In [ ]:
def gt_spectrogram(audio_path, wave_model, spec_gen_model):
    features = wave_model.process(audio_path, trim=False)
    audio, audio_length = features, torch.tensor(features.shape[0]).long()
    audio = audio.unsqueeze(0).to(device=spec_gen_model.device)
    audio_length = audio_length.unsqueeze(0).to(device=spec_gen_model.device)
    with torch.no_grad():
        spectrogram, spec_len = spec_gen_model.preprocessor(input_signal=audio, length=audio_length)
    return spectrogram, spec_len

def gen_spectrogram(text, spec_gen_model, speaker, reference_spec, reference_spec_lens):
    parsed = spec_gen_model.parse(text)
    speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
    with torch.no_grad():    
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, 
                                                          speaker=speaker, 
                                                          reference_spec=reference_spec, 
                                                          reference_spec_lens=reference_spec_lens)

    return spectrogram
  
def synth_audio(vocoder_model, spectrogram):    
    with torch.no_grad():  
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return audio

In [ ]:
# Reference Audio
reference_records = []
with open(train_manifest, "r") as f:
    for i, line in enumerate(f):
        reference_records.append(json.loads(line))

speaker_to_index = defaultdict(list)
for i, d in enumerate(reference_records): speaker_to_index[d.get('speaker', None)].append(i)
        
# Validatation Audio
num_val = 3
val_records = []
with open(valid_manifest, "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break

In [ ]:
for i, val_record in enumerate(val_records):
    reference_record = reference_records[speaker_to_index[val_record['speaker']][0]]
    reference_spec, reference_spec_lens = gt_spectrogram(reference_record['audio_filepath'], wave_model, spec_model)
    reference_spec = reference_spec.to(spec_model.device)
    spec_pred = gen_spectrogram(val_record['text'], 
                                spec_model,
                                speaker=val_record['speaker'], 
                                reference_spec=reference_spec, 
                                reference_spec_lens=reference_spec_lens)

    audio_gen = synth_audio(vocoder_model, spec_pred)
    
    audio_ref = ipd.Audio(reference_record['audio_filepath'], rate=sample_rate)
    audio_gt = ipd.Audio(val_record['audio_filepath'], rate=sample_rate)
    audio_gen = ipd.Audio(audio_gen, rate=sample_rate)
    
    print("------")
    print(f"Text: {val_record['text']}")
    print('Reference Audio')
    ipd.display(audio_ref)
    print('Ground Truth Audio')
    ipd.display(audio_gt)
    print('Synthesized Audio')
    ipd.display(audio_gen)
    plt.imshow(spec_pred[0].to('cpu').numpy(), origin="lower", aspect="auto")
    plt.show()

In [ ]:
print(f"FastPitch checkpoint: {pretrained_fastpitch_checkpoint}")
print(f"HiFi-Gan checkpoint: {finetuned_hifigan_on_multispeaker_checkpoint}")